In [1]:
import matplotlib.pyplot as plt
import numpy as np
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
  
def get_file_contents(filename):

    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

class TwitterClient(object): 

    def __init__(self): 

        consumer_key = get_file_contents("consumerKey.txt")
        consumer_secret = get_file_contents("consumerSecret.txt")
        access_token = get_file_contents("accessToken.txt")
        access_token_secret = get_file_contents("accessTokenSecret.txt")
  
        try: 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            self.auth.set_access_token(access_token, access_token_secret) 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        if analysis.sentiment.polarity > 0: 
            return 'positive', analysis.sentiment.polarity
        elif analysis.sentiment.polarity == 0: 
            return 'neutral', analysis.sentiment.polarity
        else: 
            return 'negative', analysis.sentiment.polarity
  
    def get_tweets(self, user_id, count = 10000): 
        tweets = [] 
        tweet_dates = []  
        fetched_tweets = []
        try: 
            for status in tweepy.Cursor(self.api.user_timeline, screen_name=user_id, tweet_mode="extended").items():
                fetched_tweets.append(status)
            

            for tweet in fetched_tweets: 
                parsed_tweet = {} 
                parsed_tweet['text'] = tweet.full_text
                parsed_tweet['sentiment'], parsed_tweet['polarity'] = self.get_tweet_sentiment(tweet.full_text) 
                parsed_tweet['date'] = tweet.created_at
  
                if tweet.retweet_count > 0: 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            return tweets
  
        except tweepy.TweepError as e: 
            print("Error : " + str(e))

def main(): 
    api = TwitterClient() 
    
    tweets = api.get_tweets(user_id = 'elonmusk', count = 10000) 
    tweet_dates = []
    tweet_polarity_points = []
    print("Tweet count: ", len(tweets))
    for tweet in tweets:
        print("Tweet: ", tweet, "  - Date: ", str(tweet['date']))
        tweet_dates.append(tweet['date'])
        tweet_polarity_points.append(tweet['polarity'])
    
    scatter_plot = plt.figure(figsize=(14, 8), dpi=80)
    plt.scatter(tweet_dates, tweet_polarity_points, s =.25, c = 'green')
    scatter_plot.show()
    scatter_plot.savefig('elon.png', dpi='figure')
    
    
    positive_tweet_sum = sum(status['sentiment'] == "positive" for status in tweets)
    neutral_tweet_sum = sum(status['sentiment'] == "neutral" for status in tweets)
    negative_tweet_sum = sum(status['sentiment'] == "negative" for status in tweets)
    
    
    print("Positive tweets: ", positive_tweet_sum, "(", ((positive_tweet_sum/len(tweets))*100), "%)")
    print("Neutral tweets: ", neutral_tweet_sum, "(", ((neutral_tweet_sum/len(tweets))*100), "%)" )
    print("Negative tweets: ", negative_tweet_sum, "(", ((negative_tweet_sum/len(tweets))*100), "%)")
if __name__ == "__main__": 
    # calling main function 
    main() 

Tweet count:  3245
Tweet:  {'text': '@s3xynews @patfan198010 Agreed, we’re ramping up service centers &amp; Tesla mobile service worldwide', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 4, 24, 18, 12, 53)}   - Date:  2019-04-24 18:12:53
Tweet:  {'text': '@Mike_Mitrakos More like $1000, but varies by country due to big differences in tax', 'sentiment': 'positive', 'polarity': 0.125, 'date': datetime.datetime(2019, 4, 24, 18, 11, 27)}   - Date:  2019-04-24 18:11:27
Tweet:  {'text': '@patfan198010 Considering', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 4, 24, 18, 9, 50)}   - Date:  2019-04-24 18:09:50
Tweet:  {'text': 'As mentioned before, Tesla full self-driving option price will increase after May 1. You can order online at https://t.co/46TXqRrsdr.', 'sentiment': 'positive', 'polarity': 0.35, 'date': datetime.datetime(2019, 4, 24, 18, 9, 12)}   - Date:  2019-04-24 18:09:12
Tweet:  {'text': '@Pornhub Like blowing twigs too?', 'se

Tweet:  {'text': '@marshgre @NickBre58305797 @letsrebel1 @paul_rocchini @markbspiegel @Tesla Like the movie Us. Scary … but also empathetic.', 'sentiment': 'negative', 'polarity': -0.5, 'date': datetime.datetime(2019, 3, 30, 2, 59, 36)}   - Date:  2019-03-30 02:59:36
Tweet:  {'text': '@martinengwicht @flcnhvy @Erdayastronaut @DiscoverMag @Ula No, but for sure the other way around!', 'sentiment': 'positive', 'polarity': 0.171875, 'date': datetime.datetime(2019, 3, 30, 2, 52, 21)}   - Date:  2019-03-30 02:52:21
Tweet:  {'text': '@TesLatino @mcjamez @TeslaAnswers @flcnhvy @Erdayastronaut @DiscoverMag @Tesla Also the intense low frequency vibrations that you can’t hear, but you can feel in your chest', 'sentiment': 'positive', 'polarity': 0.1, 'date': datetime.datetime(2019, 3, 30, 2, 8, 42)}   - Date:  2019-03-30 02:08:42
Tweet:  {'text': '@TeslaAnswers @flcnhvy @Erdayastronaut @DiscoverMag We’re going to try. Side boosters come back to Cape, center core lands on droneship. Latter will be

Tweet:  {'text': '@kristenhenry55 Won’t be long before they do', 'sentiment': 'negative', 'polarity': -0.05, 'date': datetime.datetime(2019, 2, 20, 0, 17, 2)}   - Date:  2019-02-20 00:17:02
Tweet:  {'text': 'Tesla made 0 cars in 2011, but will make around 500k in 2019', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 2, 20, 0, 15, 41)}   - Date:  2019-02-20 00:15:41
Tweet:  {'text': '4000 Tesla cars loading in SF for Europe https://t.co/BODbSzo3Fr', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 2, 20, 0, 2, 34)}   - Date:  2019-02-20 00:02:34
Tweet:  {'text': '@TheOnion The gritty truth …', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 2, 19, 3, 40, 3)}   - Date:  2019-02-19 03:40:03
Tweet:  {'text': 'Did meme review last night with Justin Roiland from @RickandMorty', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2019, 2, 18, 22, 4, 37)}   - Date:  2019-02-18 22:04:37
Tweet:  {'text': 'R

Tweet:  {'text': '@Kristennetten @13ericralph31 @AwakenedParadi1 @demishassabis Yes', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 12, 27, 6, 56, 55)}   - Date:  2018-12-27 06:56:55
Tweet:  {'text': '@13ericralph31 @AwakenedParadi1 @demishassabis Probability at 60% &amp; rising rapidly due to new architecture', 'sentiment': 'positive', 'polarity': 0.005681818181818177, 'date': datetime.datetime(2018, 12, 27, 6, 46, 44)}   - Date:  2018-12-27 06:46:44
Tweet:  {'text': '@AwakenedParadi1 @demishassabis 30%', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 12, 27, 6, 34, 11)}   - Date:  2018-12-27 06:34:11
Tweet:  {'text': '@demishassabis Congratulations', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 12, 27, 6, 17, 17)}   - Date:  2018-12-27 06:17:17
Tweet:  {'text': '@ckgaparajita Coming soon', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 12, 27, 4, 3, 45)}   - Date:  2018-12-27 04

Tweet:  {'text': '@SeanGVarney That is true &amp; should be applauded. Right move is for oil companies truly to think of themselves as energy companies, but move rapidly towards clean, sustainable energy. Means lower profits short-term, but will make them stronger long-term.', 'sentiment': 'positive', 'polarity': 0.3341269841269841, 'date': datetime.datetime(2018, 11, 10, 21, 26, 56)}   - Date:  2018-11-10 21:26:56
Tweet:  {'text': 'We know we’ll run out of dead dinosaurs to mine for fuel &amp; have to use sustainable energy eventually, so why not go renewable now &amp; avoid increasing risk of climate catastrophe? Betting that science is wrong &amp; oil companies are right is the dumbest experiment in history by far … https://t.co/TvyuDBf3lR', 'sentiment': 'negative', 'polarity': -0.07857142857142857, 'date': datetime.datetime(2018, 11, 10, 21, 19, 43)}   - Date:  2018-11-10 21:19:43
Tweet:  {'text': 'RT @AndreiBulu: Horrible Bay Area air quality due to Paradise, CA fire...  Check out

Tweet:  {'text': 'Not easy coming up with a good cover story for an underground volcano lair, but I think it’s working', 'sentiment': 'positive', 'polarity': 0.24166666666666664, 'date': datetime.datetime(2018, 9, 8, 21, 20, 46)}   - Date:  2018-09-08 21:20:46
Tweet:  {'text': 'Test-driving our new boring machine with a 🎮 https://t.co/WBcDo8ra6p', 'sentiment': 'negative', 'polarity': -0.4318181818181818, 'date': datetime.datetime(2018, 9, 8, 20, 36, 13)}   - Date:  2018-09-08 20:36:13
Tweet:  {'text': '@Tesla_Bill @InsideEVs 2170 is about the right cell size, but I don’t recommend copying our current module design. Far too difficult to manufacture.', 'sentiment': 'negative', 'polarity': -0.02857142857142858, 'date': datetime.datetime(2018, 9, 8, 9, 34, 39)}   - Date:  2018-09-08 09:34:39
Tweet:  {'text': '@caitylotz Like sands through the hourglass …', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 9, 8, 7, 43, 48)}   - Date:  2018-09-08 07:43:48
Tweet:  {'tex

Tweet:  {'text': '@JaneidyEve Yes. Those videos are from this morning.', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 7, 9, 0, 45, 25)}   - Date:  2018-07-09 00:45:25
Tweet:  {'text': '@Anna_online According to divers who have made the passage, yes. However, we also made an exact replica that is inflatable, so that the entire path can be tested without risk of blockage.', 'sentiment': 'positive', 'polarity': 0.125, 'date': datetime.datetime(2018, 7, 9, 0, 39, 46)}   - Date:  2018-07-09 00:39:46
Tweet:  {'text': 'Simulating maneuvering through a narrow passage https://t.co/2z01Ut3vxJ', 'sentiment': 'negative', 'polarity': -0.2, 'date': datetime.datetime(2018, 7, 9, 0, 4, 30)}   - Date:  2018-07-09 00:04:30
Tweet:  {'text': '@2morrowknight Thanks, but we’ve not done anything useful yet. It is all other people.', 'sentiment': 'positive', 'polarity': 0.125, 'date': datetime.datetime(2018, 7, 8, 23, 16, 2)}   - Date:  2018-07-08 23:16:02
Tweet:  {'text': '@bweikl

Tweet:  {'text': '@NickAPappas Sigh', 'sentiment': 'neutral', 'polarity': 0.0, 'date': datetime.datetime(2018, 6, 16, 15, 51, 31)}   - Date:  2018-06-16 15:51:31
Tweet:  {'text': 'By the way, I am actually a socialist. Just not the kind that shifts resources from most productive to least productive, pretending to do good, while actually causing harm. True socialism seeks greatest good for all.', 'sentiment': 'positive', 'polarity': 0.39444444444444443, 'date': datetime.datetime(2018, 6, 16, 15, 47, 8)}   - Date:  2018-06-16 15:47:08
Tweet:  {'text': '“No sense of humor” is certainly proving itself true. Good grief!\nHow many socialists does it take to screw in a lightbulb? \nAnswer: That’s not funny!!', 'sentiment': 'positive', 'polarity': 0.09482886904761904, 'date': datetime.datetime(2018, 6, 16, 15, 43, 55)}   - Date:  2018-06-16 15:43:55
Tweet:  {'text': '@R850Mango @tinytacotown Boys High is a government school for anyone local, which we were', 'sentiment': 'positive', 'polarity':

Positive tweets:  1532 ( 47.21109399075501 %)
Neutral tweets:  1291 ( 39.78428351309707 %)
Negative tweets:  422 ( 13.004622496147919 %)


C:\Users\Jack\Anaconda3\lib\site-packages\matplotlib\figure.py:445: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
